In [75]:
import pandas as pd
import ssl
import matplotlib.pyplot as plt
import numpy as np
import hashlib
import boto3

In [76]:
import warnings
warnings.filterwarnings("ignore")

In [77]:
pd.set_option('display.max_columns', 500)

In [78]:
ssl._create_default_https_context = ssl._create_unverified_context

# Tooling

### Upload S3

In [79]:
def UploadS3(filepath, key):
    s3 = boto3.client("s3")
    bucket_name = "economia-popular-delivery-content-indices"
    s3.upload_file(
        Filename=filepath,
        Bucket=bucket_name,
        Key=key,
        ExtraArgs={
            "ACL": "public-read",
            "ServerSideEncryption": "AES256"
        }
    )

    return True

### Hash String

In [80]:
def create_hash(text):
    return hashlib.sha256(text.encode()).hexdigest()

### IBGE parser

In [81]:
def ibge_dataframe_builder(df, items, classification, year):
    year = "{}".format(year)
    for item in items:
        dimensao = item["localidade"]["nivel"]["nome"]
        localidade = item["localidade"]["nome"]
        populacao = item["serie"]["{}".format(year)]
        id = create_hash(year + "-" + dimensao + "-" + localidade)
        
        row = {
            "id": id,
            "ano": year,
            "dimensao": dimensao,
            "localidade": localidade,
            "cor_raca": classification, 
            "populacao": populacao
        }
        df = df.append(row, ignore_index=True)        
    return df

## Dataframes

### Criando a estrutura base do Dataframe Vazia

In [82]:
columns = ["id", "ano", "dimensao", "localidade" , "cor_raca", "populacao"]
df = pd.DataFrame(columns=columns)
print(df)

Empty DataFrame
Columns: [id, ano, dimensao, localidade, cor_raca, populacao]
Index: []


In [83]:
columns_consolidado = ["id", "ano", "dimensao", "localidade" , "populacao_branca", "populacao_negra", "populacao_amarela", "populacao_parda","populacao_indigena" ,"populacao_sem_declaracao" ,"populacao_total"]
df_consolidado = pd.DataFrame(columns=columns_consolidado)
print(df_consolidado)

Empty DataFrame
Columns: [id, ano, dimensao, localidade, populacao_branca, populacao_negra, populacao_amarela, populacao_parda, populacao_indigena, populacao_sem_declaracao, populacao_total]
Index: []


In [84]:
# df = pd.read_json("https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7%5Ball%5D%7CN3%5Ball%5D&classificacao=86%5Ball%5D%7C12056%5B99566%5D%7C58%5B0%5D")

In [85]:
df.head()

,id,ano,dimensao,localidade,cor_raca,populacao


In [86]:
df.tail()

,id,ano,dimensao,localidade,cor_raca,populacao


In [87]:
df.shape

(0, 6)

### URL's de Consulta no IBGE

In [88]:
brasil_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[all]|12056[99566]"
total_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[0]|12056[99566]|58[0]"
branca_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2776]|12056[99566]|58[0]"
preta_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2777]|12056[99566]|58[0]"
amarela_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2778]|12056[99566]|58[0]" 
parda_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2779]|12056[99566]|58[0]"
indigena_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2780]|12056[99566]|58[0]"
sem_declaracao_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2781]|12056[99566]|58[0]"

### População Brasil

In [ ]:
df_temp = pd.read_json(brasil_url)

In [ ]:
series = df_temp.resultados[0][0]['series']

In [ ]:
df = ibge_dataframe_builder(df, series, "brasil", 2010)

### População Total

In [89]:
df_temp = pd.read_json(total_url)

In [90]:
series = df_temp.resultados[0][0]['series']

In [91]:
df = ibge_dataframe_builder(df, series, "total", 2010)

In [92]:
df.shape

(63, 6)

### População Declarada como Branca

In [93]:
df_temp = pd.read_json(branca_url)

In [94]:
series = df_temp.resultados[0][0]['series']

In [95]:
df = ibge_dataframe_builder(df, series, "branca", 2010)

In [96]:
df.shape

(126, 6)

### População declarada como preta

In [97]:
df_temp = pd.read_json(preta_url)

In [98]:
series = df_temp.resultados[0][0]['series']

In [99]:
df = ibge_dataframe_builder(df, series, "preta", 2010)

In [100]:
df.shape

(189, 6)

### Populacão declarada como amarela

In [101]:
df_temp = pd.read_json(amarela_url)

In [102]:
series = df_temp.resultados[0][0]['series']

In [103]:
df = ibge_dataframe_builder(df, series, "amarela", 2010)

In [104]:
df.shape

(252, 6)

### População Parda

In [105]:
df_temp = pd.read_json(parda_url)

In [106]:
series = df_temp.resultados[0][0]['series']

In [107]:
df = ibge_dataframe_builder(df, series, "parda", 2010)

In [108]:
df.shape

(315, 6)

### População Indigena

In [109]:
df_temp = pd.read_json(indigena_url)

In [110]:
series = df_temp.resultados[0][0]['series']

In [111]:
df = ibge_dataframe_builder(df, series, "indigena", 2010)

In [112]:
df.shape

(378, 6)

### População Sem Declaração

In [113]:
df_temp = pd.read_json(sem_declaracao_url)

In [114]:
series = df_temp.resultados[0][0]['series']

In [115]:
df = ibge_dataframe_builder(df, series, "sem_declaracao", 2010)

In [116]:
df.shape

(441, 6)

## Sumário Exploratório

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          441 non-null    object
 1   ano         441 non-null    object
 2   dimensao    441 non-null    object
 3   localidade  441 non-null    object
 4   cor_raca    441 non-null    object
 5   populacao   441 non-null    object
dtypes: object(6)
memory usage: 20.8+ KB


In [118]:
df.head()

,id,ano,dimensao,localidade,cor_raca,populacao
0,5ce1397542c815696a781695e1064515dcc99a2f76dc13...,2010,Região Metropolitana,Manaus - AM,total,966485
1,225ca007c01f71a0bfdc0ca7dbe77c26e9424c109f1573...,2010,Região Metropolitana,Belém - PA,total,989834
2,a680f96119c12ab0d4629f77563b6bb340d456ebb21499...,2010,Região Metropolitana,Macapá - AP,total,227515
3,cf0c0c0e84255199f73093a9352d5c209c32d71128ff39...,2010,Região Metropolitana,Grande São Luís - MA,total,639530
4,7491f56685d005e0aca565ddfefa6f85fae1212714102d...,2010,Região Metropolitana,Sudoeste Maranhense - MA,total,155588


In [119]:
df.tail()

,id,ano,dimensao,localidade,cor_raca,populacao
436,d23860d67f080e8454155455c2cc47c34f2637b24ba27f...,2010,Unidade da Federação,Rio Grande do Sul,sem_declaracao,34
437,29cc530b8162c0ff5f37a34a07dc7f2dc71e212ef73917...,2010,Unidade da Federação,Mato Grosso do Sul,sem_declaracao,-
438,f9a89474f5ba626244505fa48a4e9bbf243418ecfbe0ff...,2010,Unidade da Federação,Mato Grosso,sem_declaracao,-
439,3991c5a6b9fc6bf8aac55aa0cd131e551d2a6e4405ac8e...,2010,Unidade da Federação,Goiás,sem_declaracao,13
440,be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc46870...,2010,Unidade da Federação,Distrito Federal,sem_declaracao,-


In [120]:
df.cor_raca.value_counts()

total             63
branca            63
preta             63
amarela           63
parda             63
indigena          63
sem_declaracao    63
Name: cor_raca, dtype: int64

In [121]:
df.id.value_counts()

5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5    7
0a6a8696b632164b70b0fdf5fce986961096c1a6bea64197046619a22388e18a    7
9a2eef2446115a88e99b7d170733f20c54f5de3ad5ec8799ca360ea2f0772ab0    7
f8102abf1ca27828da51f064f38f708ba345e30552cef85dc920fe3e0b8a4dcd    7
08fb0d5a23679a57d8b8c8cc3875dbc7f062c58a881f4c87cb9ea3af95ed6169    7
                                                                   ..
1ee5cc191a2d21acde5f9faccbe4dde5116e6d3b19502b389ebaa072c41f27ee    7
5cd9f04b5a5280cabfa0ee7ee7765b73d6ce503bd69c4f182855286d336e74b6    7
29df00795eeae28e5ab5b69ee2205119d8e2fc77adff9812882b443014449803    7
2519d535b130812199cd1b90993048e9ae6669ccd5da8af0ee300231a0c00cc1    7
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    7
Name: id, Length: 63, dtype: int64

# Consolidação dos Dados

## Criando o DF de referencia

In [122]:
df_consolidado.id = df[["id"]].copy()

In [123]:
df_consolidado.head()

,id,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
0,5ce1397542c815696a781695e1064515dcc99a2f76dc13...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,225ca007c01f71a0bfdc0ca7dbe77c26e9424c109f1573...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a680f96119c12ab0d4629f77563b6bb340d456ebb21499...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cf0c0c0e84255199f73093a9352d5c209c32d71128ff39...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7491f56685d005e0aca565ddfefa6f85fae1212714102d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
df_consolidado.id.value_counts()

5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5    7
0a6a8696b632164b70b0fdf5fce986961096c1a6bea64197046619a22388e18a    7
9a2eef2446115a88e99b7d170733f20c54f5de3ad5ec8799ca360ea2f0772ab0    7
f8102abf1ca27828da51f064f38f708ba345e30552cef85dc920fe3e0b8a4dcd    7
08fb0d5a23679a57d8b8c8cc3875dbc7f062c58a881f4c87cb9ea3af95ed6169    7
                                                                   ..
1ee5cc191a2d21acde5f9faccbe4dde5116e6d3b19502b389ebaa072c41f27ee    7
5cd9f04b5a5280cabfa0ee7ee7765b73d6ce503bd69c4f182855286d336e74b6    7
29df00795eeae28e5ab5b69ee2205119d8e2fc77adff9812882b443014449803    7
2519d535b130812199cd1b90993048e9ae6669ccd5da8af0ee300231a0c00cc1    7
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    7
Name: id, Length: 63, dtype: int64

In [125]:
df_consolidado = df_consolidado.drop_duplicates(subset='id', keep='first')

In [126]:
df_consolidado.id.value_counts()

5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5    1
0a6a8696b632164b70b0fdf5fce986961096c1a6bea64197046619a22388e18a    1
9a2eef2446115a88e99b7d170733f20c54f5de3ad5ec8799ca360ea2f0772ab0    1
f8102abf1ca27828da51f064f38f708ba345e30552cef85dc920fe3e0b8a4dcd    1
08fb0d5a23679a57d8b8c8cc3875dbc7f062c58a881f4c87cb9ea3af95ed6169    1
                                                                   ..
1ee5cc191a2d21acde5f9faccbe4dde5116e6d3b19502b389ebaa072c41f27ee    1
5cd9f04b5a5280cabfa0ee7ee7765b73d6ce503bd69c4f182855286d336e74b6    1
29df00795eeae28e5ab5b69ee2205119d8e2fc77adff9812882b443014449803    1
2519d535b130812199cd1b90993048e9ae6669ccd5da8af0ee300231a0c00cc1    1
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    1
Name: id, Length: 63, dtype: int64

In [127]:
df_consolidado.index = df_consolidado.id

In [128]:
df_consolidado = df_consolidado.drop(columns="id", axis=1)

In [129]:
df_consolidado.head()

,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
id,,,,,,,,,,
5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225ca007c01f71a0bfdc0ca7dbe77c26e9424c109f157378bad9b2605f7867bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a680f96119c12ab0d4629f77563b6bb340d456ebb214990360faaebf9f9f00b2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cf0c0c0e84255199f73093a9352d5c209c32d71128ff395a2c26d163c0bfd235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7491f56685d005e0aca565ddfefa6f85fae1212714102db8eb066d4104c05274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Inputando os dados quantitativos da população

In [130]:
for index, row in df.iterrows():
    # print(row.id)
    df_consolidado.loc[row.id, "ano"] = row.ano
    df_consolidado.loc[row.id, "dimensao"] = row.dimensao
    df_consolidado.loc[row.id, "localidade"] = row.localidade

    if row.cor_raca == "total":
        df_consolidado.loc[row.id, "populacao_total"] = row.populacao
    elif row.cor_raca == "branca":
        df_consolidado.loc[row.id, "populacao_branca"] = row.populacao
    elif row.cor_raca == "preta":
        df_consolidado.loc[row.id, "populacao_negra"] = row.populacao
    elif row.cor_raca == "amarela":
        df_consolidado.loc[row.id, "populacao_amarela"] = row.populacao    
    elif row.cor_raca == "parda":
        df_consolidado.loc[row.id, "populacao_parda"] = row.populacao 
    elif row.cor_raca == "indigena":
        df_consolidado.loc[row.id, "populacao_indigena"] = row.populacao
    elif row.cor_raca == "sem_declaracao":
        df_consolidado.loc[row.id, "populacao_sem_declaracao"] = row.populacao 
    # break

In [131]:
df_consolidado.replace("-", 0, inplace=True)

### Convertendo as colunas para integer

In [132]:
df_consolidado["populacao_total"] = df_consolidado["populacao_total"].astype(int)
df_consolidado["populacao_branca"] = df_consolidado["populacao_branca"].astype(int)
df_consolidado["populacao_negra"] = df_consolidado["populacao_negra"].astype(int)
df_consolidado["populacao_amarela"] = df_consolidado["populacao_amarela"].astype(int)
df_consolidado["populacao_parda"] = df_consolidado["populacao_parda"].astype(int)
df_consolidado["populacao_indigena"] = df_consolidado["populacao_indigena"].astype(int)
df_consolidado["populacao_sem_declaracao"] = df_consolidado["populacao_sem_declaracao"].astype(int)

### Conferencia da Consolidação Quantitativa

In [133]:
df_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 5ce1397542c815696a781695e1064515dcc99a2f76dc135a4ce586f19bd1e2d5 to be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ano                       63 non-null     object
 1   dimensao                  63 non-null     object
 2   localidade                63 non-null     object
 3   populacao_branca          63 non-null     int64 
 4   populacao_negra           63 non-null     int64 
 5   populacao_amarela         63 non-null     int64 
 6   populacao_parda           63 non-null     int64 
 7   populacao_indigena        63 non-null     int64 
 8   populacao_sem_declaracao  63 non-null     int64 
 9   populacao_total           63 non-null     int64 
dtypes: int64(7), object(3)
memory usage: 7.5+ KB


In [134]:
df_consolidado.iloc[1]

ano                                         2010
dimensao                    Região Metropolitana
localidade                            Belém - PA
populacao_branca                          247878
populacao_negra                            84605
populacao_amarela                           8441
populacao_parda                           647451
populacao_indigena                          1460
populacao_sem_declaracao                       0
populacao_total                           989834
Name: 225ca007c01f71a0bfdc0ca7dbe77c26e9424c109f157378bad9b2605f7867bb, dtype: object

## Criando as porcentagens

In [135]:
df_consolidado['populacao_branca_porcentagem'] = ((df_consolidado['populacao_branca'] / df_consolidado['populacao_total']) * 100).round(2) 

In [136]:
df_consolidado['populacao_negra_porcentagem'] = ((df_consolidado['populacao_negra'] / df_consolidado['populacao_total']) * 100).round(2) 

In [137]:
df_consolidado['populacao_amarela_porcentagem'] = ((df_consolidado['populacao_amarela'] / df_consolidado['populacao_total']) * 100).round(2) 

In [138]:
df_consolidado['populacao_parda_porcentagem'] = ((df_consolidado['populacao_parda'] / df_consolidado['populacao_total']) * 100).round(2) 

In [139]:
df_consolidado['populacao_indigena_porcentagem'] = ((df_consolidado['populacao_indigena'] / df_consolidado['populacao_total']) * 100).round(2) 

In [140]:
df_consolidado['populacao_sem_declaracao_porcentagem'] = ((df_consolidado['populacao_sem_declaracao'] / df_consolidado['populacao_total']) * 100).round(2) 

In [141]:
df_consolidado.iloc[1]

ano                                                     2010
dimensao                                Região Metropolitana
localidade                                        Belém - PA
populacao_branca                                      247878
populacao_negra                                        84605
populacao_amarela                                       8441
populacao_parda                                       647451
populacao_indigena                                      1460
populacao_sem_declaracao                                   0
populacao_total                                       989834
populacao_branca_porcentagem                           25.04
populacao_negra_porcentagem                             8.55
populacao_amarela_porcentagem                           0.85
populacao_parda_porcentagem                            65.41
populacao_indigena_porcentagem                          0.15
populacao_sem_declaracao_porcentagem                     0.0
Name: 225ca007c01f71a0bf

# Upload

## Salvando os Dados

In [142]:
df_consolidado.to_json('datasets/pessoas_economicamente_ativas.json', orient='records', lines=True)

In [143]:
df_consolidado.to_csv('datasets/pessoas_economicamente_ativas.csv', index=False)

In [144]:
UploadS3('datasets/pessoas_economicamente_ativas.json', 'sociais/pessoas_economicamente_ativas.pessoas_economicamente_ativas.json')

True

In [145]:
UploadS3('datasets/pessoas_economicamente_ativas.csv', 'sociais/pessoas_economicamente_ativas/pessoas_economicamente_ativas.csv')

True